Configure Database

In [ ]:
from helpers.config import Config
from sqlalchemy import create_engine

sql_engine = create_engine(f"{Config.POSTGRES_CONNECTION_STRING}/sql_agent")

Ingest Data

In [ ]:
import pandas as pd
from helpers import get_vector_store_instance
from langchain_core.documents import Document
from helpers import load_excel

data = load_excel("./sample_data/brillar_bank_fixed_deposit.xlsx")
df = pd.read_excel("./sample_data/brillar_bank_fixed_deposit.xlsx", sheet_name=None)


vector_store = get_vector_store_instance(embedding_model="text-embedding-3-large", dimension=256, index_name="csv_data", vector_db="qdrant")
vector_store.add_documents(data)
for sheet_name, df in df.items():
    table_name = sheet_name.replace(" ", "_").lower()
    df.to_sql(name=table_name, con=sql_engine, if_exists="replace", index=False)
    print(f"Inserted data from sheet '{sheet_name}' into table '{table_name}'")
    

Initiate LLM

In [5]:
from helpers import get_llm

llm = get_llm("gpt-4o")

In [6]:
from helpers.conversation_retrieval_chain import sql_chain_with_embedding_router, sql_chain_with_llm_router, sql_chain_with_two_store
from helpers import get_retriever

retriever = get_retriever(index_name="test", embedding_model="text-embedding-3-large", dimension=256, vector_db="qdrant")
llm_router_chain = sql_chain_with_llm_router(llm=llm, retriever=retriever, sql_engine=sql_engine)
embedding_router_chain = sql_chain_with_embedding_router(llm=llm, retriever=retriever, sql_engine=sql_engine)
two_store_chain = sql_chain_with_two_store(llm=llm, retriever=retriever, sql_engine=sql_engine)

Invoke the Chain

In [ ]:
question = "What are the interest rates for fixed deposit?"
print(f'Question: {question}')
print("-------------------")
result1 = embedding_router_chain.invoke(
    {
        "question": question,
        "chat_history": [],
    }
    # , config={"callbacks": [ConsoleCallbackHandler()]}
)
print(f"\n{result1}\n")
print("-------------------")
result2 = llm_router_chain.invoke(
    {
        "question": question,
        "chat_history": [],
    }
    # , config={"callbacks": [ConsoleCallbackHandler()]}
)
print(f"\n{result2}")

In [13]:
from langchain.callbacks.tracers import ConsoleCallbackHandler

question = "who has the most amount of deposit?"
print(f'Question: {question}')
print("-------------------")
result1 = two_store_chain.invoke(
    {
        "question": question,
        "chat_history": [],
    }
    # , config={"callbacks": [ConsoleCallbackHandler()]}
)
result1

Question: who has the most amount of deposit?
-------------------
LLM Router.
Routing to SQL Chain.


'Ethan Chan has the most amount of deposit, with RM 200,000.'